## **Implement Advanced ML Algorithm for Classification**
**Dataset from BlackBoard:** Facial Emotion Recognition dataset

**File:** facial-emotion-recognition.zip (432.768 MB)

### **ToDo**
- Model eval: use the test set from intel
- Try ResNet model
- Hyperparameter tuning
- Use proper loss/metrics standard in image classification
- Include early stopping
- Double check approach for base model fine tuning
- Comparability: use same imported metric on all algorithms
- Tag images with confidence %
- Explain model: google/vit-base-patch16-224-in21k
- Here we interpret term advanced according to lecture slides, which focus on knowledge transfer.
- In modern image classification goes without required pre-processing, thus skipped here. Traditional models do require manual crafted feature enginnering, while one of the key properties of ANN-based, feature engineering is inherent within the model as input data is transformed between hidden layers.
- Explain the code
- There are many base models to select from, here we selected visual transformer (ViT) as these are considered SOTA.



@misc{wu2020visual,
      title={Visual Transformers: Token-based Image Representation and Processing for Computer Vision}, 
      author={Bichen Wu and Chenfeng Xu and Xiaoliang Dai and Alvin Wan and Peizhao Zhang and Zhicheng Yan and Masayoshi Tomizuka and Joseph Gonzalez and Kurt Keutzer and Peter Vajda},
      year={2020},
      eprint={2006.03677},
      archivePrefix={arXiv},
      primaryClass={cs.CV}
}

In [2]:
# Step 1: Define the path to your local dataset
dataset_path = "/Users/thomas/Desktop/IT3212/assignment_4/thomas/intel-image-classification/seg_train/seg_train/"  # Replace this with the actual path to your dataset

In [3]:
import os
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor
from torch import nn, optim
from sklearn.metrics import classification_report
from collections import Counter

# Step 1: Define the path to your local dataset
dataset_path = "/Users/thomas/Desktop/IT3212/assignment_4/thomas/intel-image-classification/seg_train/seg_train/"  # Replace this with the actual path to your dataset

# Step 1: Load the full dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the expected input size for ViT
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet standard normalization
]))

# Ensure that we correctly detect the classes from the subfolders
class_names = full_dataset.classes  # List of class names (subfolder names under dataset_path)
print("Classes found in the dataset:")
for idx, class_name in enumerate(class_names):
    print(f"{idx}: {class_name}")

# Step 2: Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Step 3: Load the pre-trained ViT model
model_name = "google/vit-base-patch16-224-in21k"  # You can try other models too
model = ViTForImageClassification.from_pretrained(model_name, num_labels=len(class_names))  # Use len(class_names) for num_labels
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

# Step 4: Set up DataLoader for training and validation
batch_size = 256
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Step 5: Print the size of training and validation datasets
print(f"\nTraining dataset size: {len(train_dataset)} samples")
print(f"Validation dataset size: {len(val_dataset)} samples")

# Step 6: Print a few examples from the dataset to verify the structure
print("\nExample images and labels from the dataset:")
for i in range(5):
    image, label = full_dataset[i]  # Access the first 5 samples
    print(f"Image {i}: Label = {class_names[label]}")  # Print class name for label


# Step 5: Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Step 6: Fine-tuning the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 15

# Variables to track loss and accuracy
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

print(f"-------- START TRAINING -----------")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    bcount = 0
    # Training loop
    for batch in train_dataloader:
        print("bcount = ", bcount)
        bcount += 1
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.logits, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    # Calculate average loss and accuracy for the epoch
    avg_loss = running_loss / len(train_dataloader)
    accuracy = 100 * correct_predictions / total_predictions

    train_loss_history.append(avg_loss)
    train_acc_history.append(accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%")

    # Step 7: Evaluate on Validation Set
    model.eval()
    val_running_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0
    with torch.no_grad():
        for batch in val_dataloader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs.logits, labels)

            val_running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.logits, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)

    # Calculate average validation loss and accuracy
    val_avg_loss = val_running_loss / len(val_dataloader)
    val_accuracy = 100 * val_correct_predictions / val_total_predictions

    val_loss_history.append(val_avg_loss)
    val_acc_history.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_avg_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

# Step 8: Plot Training and Validation Loss and Accuracy
plt.figure(figsize=(12, 6))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(train_loss_history, label='Training Loss')
plt.plot(val_loss_history, label='Validation Loss', linestyle='--')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_acc_history, label='Training Accuracy')
plt.plot(val_acc_history, label='Validation Accuracy', linestyle='--')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()

# Step 9: Final Evaluation on Validation Set
model.eval()
predictions = []
true_labels = []

# Ensure that the evaluation is done using the validation DataLoader
for batch in val_dataloader:
    images, labels = batch
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(images)

    preds = torch.argmax(outputs.logits, dim=1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Classification report
print(classification_report(true_labels, predictions, target_names=full_dataset.classes))


Classes found in the dataset:
0: buildings
1: forest
2: glacier
3: mountain
4: sea
5: street


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/thomas/miniconda3/envs/.machine_learning/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(



Training dataset size: 136 samples
Validation dataset size: 34 samples

Example images and labels from the dataset:
Image 0: Label = buildings
Image 1: Label = buildings
Image 2: Label = buildings
Image 3: Label = buildings
Image 4: Label = buildings
-------- START TRAINING -----------
bcount =  0
Epoch [1/15], Train Loss: 1.7717, Train Accuracy: 21.32%
Epoch [1/15], Val Loss: 1.7500, Val Accuracy: 26.47%
bcount =  0
Epoch [2/15], Train Loss: 1.7520, Train Accuracy: 30.88%
Epoch [2/15], Val Loss: 1.7370, Val Accuracy: 29.41%
bcount =  0
Epoch [3/15], Train Loss: 1.7328, Train Accuracy: 41.18%
Epoch [3/15], Val Loss: 1.7240, Val Accuracy: 35.29%
bcount =  0
Epoch [4/15], Train Loss: 1.7137, Train Accuracy: 53.68%
Epoch [4/15], Val Loss: 1.7109, Val Accuracy: 41.18%
bcount =  0
Epoch [5/15], Train Loss: 1.6948, Train Accuracy: 61.76%
Epoch [5/15], Val Loss: 1.6978, Val Accuracy: 55.88%
bcount =  0
Epoch [6/15], Train Loss: 1.6758, Train Accuracy: 65.44%
Epoch [6/15], Val Loss: 1.6847, V

KeyboardInterrupt: 

---
### ResNet
---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.optim import lr_scheduler
import os
import time
import copy
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torch import nn, optim
from sklearn.metrics import classification_report
from collections import Counter

# Step 1: Define the path to your local dataset
dataset_path = "/Users/thomas/Desktop/IT3212/assignment_4/thomas/intel-image-classification/seg_train/seg_train/"  # Replace this with the actual path to your dataset

# Step 1: Load the full dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the expected input size for ViT
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet standard normalization
]))

# Ensure that we correctly detect the classes from the subfolders
class_names = full_dataset.classes  # List of class names (subfolder names under dataset_path)
print("Classes found in the dataset:")
for idx, class_name in enumerate(class_names):
    print(f"{idx}: {class_name}")

# Step 2: Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


# Step 4: Set up DataLoader for training and validation
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Step 5: Print the size of training and validation datasets
print(f"\nTraining dataset size: {len(train_dataset)} samples")
print(f"Validation dataset size: {len(val_dataset)} samples")

# Step 6: Print a few examples from the dataset to verify the structure
print("\nExample images and labels from the dataset:")
for i in range(5):
    image, label = full_dataset[i]  # Access the first 5 samples
    print(f"Image {i}: Label = {class_names[label]}")  # Print class name for label


#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
# Load a pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze the parameters in the pre-trained model (optional)
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer to match the number of classes in your dataset
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))  # Modify output layer to match the number of classes

# If you want to fine-tune the whole model, you can unfreeze the last few layers
# Unfreeze the last block
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define Loss function (CrossEntropyLoss for multi-class classification)
criterion = nn.CrossEntropyLoss()

# Define Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Learning rate scheduler (optional, helps with fine-tuning)
#scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Function to train the model
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluation mode
            
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data
            dataloaders = train_loader if phase == 'train' else val_loader
            for inputs, labels in dataloaders:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)#

            #if phase == 'train':
            #    scheduler.step()

            epoch_loss = running_loss / len(dataloaders.dataset)
            epoch_acc = running_corrects.double() / len(dataloaders.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # Deep copy the model if it's the best
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60}m {time_elapsed % 60}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=10)

# Evaluate the model on the validation set
def evaluate_model(model, dataloader):
    model.eval()  # Set model to evaluation mode
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloader.dataset)
    print(f'Validation Accuracy: {accuracy:.4f}')
    
evaluate_model(model, val_loader)



In [ ]:
import numpy as np

# Step 10: Display some test images with predicted and ground truth labels
# Pick a few test images for visualization
num_images = 8
fig, axes = plt.subplots(1, num_images, figsize=(15, 3))

model.eval()  # Set the model to evaluation mode

for i, (images, labels) in enumerate(test_dataloader):
    print(i, num_images)
    if i >= num_images:  # Limit to num_images
        print('breaking')
        break
    images, labels = images.to(device), labels.to(device)
    
    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs.logits, 1)
        
        # Convert tensor to numpy for plotting
        image = images[0].cpu().numpy().transpose((1, 2, 0))  # CxHxW -> HxWxC
        image = (image * 0.229 + 0.485)  # Reverse the normalization
        image = np.clip(image, 0, 1)  # Ensure values are between 0 and 1

        # Get the ground truth label
        true_label = full_dataset.classes[labels[0].item()]

        # Plot the image with both ground truth and predicted label in the title
        axes[i].imshow(image)
        axes[i].axis('off')  # Remove axes
        axes[i].set_title(f"GT: {true_label}\nPred: {full_dataset.classes[predicted[0]]}", fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
# Step 11: Dense Plot of All Training Images
# Create a dense grid of all training images
num_train_images = len(train_dataset)
fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(15, 15))  # Adjust the number of rows and columns as needed
axes = axes.flatten()

for i in range(num_train_images):
    image, label = train_dataset[i]
    image = image.cpu().numpy().transpose((1, 2, 0))  # CxHxW -> HxWxC
    image = (image * 0.229 + 0.485)  # Reverse the normalization
    image = np.clip(image, 0, 1)  # Ensure values are between 0 and 1
    
    axes[i].imshow(image)
    axes[i].axis('off')
    axes[i].set_title(full_dataset.classes[label], fontsize=8)

plt.tight_layout()
plt.suptitle('Training Images', fontsize=16)
plt.subplots_adjust(top=0.95)
plt.show()

# Step 12: Dense Plot of All Test Images
# Create a dense grid of all test images
num_test_images = len(test_dataset)
fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(15, 15))  # Adjust the number of rows and columns as needed
axes = axes.flatten()

for i in range(num_test_images):
    image, label = test_dataset[i]
    image = image.cpu().numpy().transpose((1, 2, 0))  # CxHxW -> HxWxC
    image = (image * 0.229 + 0.485)  # Reverse the normalization
    image = np.clip(image, 0, 1)  # Ensure values are between 0 and 1
    
    axes[i].imshow(image)
    axes[i].axis('off')
    axes[i].set_title(full_dataset.classes[label], fontsize=8)

plt.tight_layout()
plt.suptitle('Test Images', fontsize=16)
plt.subplots_adjust(top=0.95)
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
from transformers import ViTForImageClassification, ViTFeatureExtractor
from sklearn.metrics import classification_report

# Step 1: Load the dataset
#dataset_path = "/path/to/your/local/dataset"  # Replace this with the actual path to your dataset

# Assuming images are organized by class in folders
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the expected input size for ViT
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
]))

# Step 2: Split dataset into train, validation, and test sets
train_size = int(0.8 * len(full_dataset))  # 80% training
test_size = len(full_dataset) - train_size  # 20% test
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Further split the training set into train and validation (10% validation)
train_size = int(0.9 * len(train_dataset))  # 90% for training
val_size = len(train_dataset) - train_size  # 10% for validation
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Step 3: Set up DataLoaders for training, validation, and testing
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Step 4: Load the pre-trained ViT model
model_name = "google/vit-base-patch16-224-in21k"
model = ViTForImageClassification.from_pretrained(model_name, num_labels=len(full_dataset.classes))
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

# Step 5: Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Step 6: Early Stopping Parameters
patience = 3  # Number of epochs to wait for improvement in validation loss before stopping
min_epochs_after_no_improvement = 5  # Minimum number of epochs to wait after no improvement before stopping
best_val_loss = float('inf')
epochs_without_improvement = 0

# Step 7: Training loop with Early Stopping
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 15  # Max number of epochs

train_loss_history = []
val_loss_history = []
train_acc_history = []
val_acc_history = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    # Training loop
    for batch in train_dataloader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.logits, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    # Calculate average loss and training accuracy for the epoch
    avg_train_loss = running_loss / len(train_dataloader)
    train_accuracy = 100 * correct_predictions / total_predictions

    train_loss_history.append(avg_train_loss)
    train_acc_history.append(train_accuracy)

    # Evaluate on the validation set
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0

    with torch.no_grad():
        for batch in val_dataloader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs.logits, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.logits, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = 100 * val_correct_predictions / val_total_predictions

    val_loss_history.append(avg_val_loss)
    val_acc_history.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping logic with minimum epochs after no improvement
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_without_improvement = 0
        # Save the model with the best validation loss
        torch.save(model.state_dict(), "best_model.pth")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"No improvement in validation loss for {patience} epochs. "
                  f"Waiting for {min_epochs_after_no_improvement} more epochs before stopping...")
            
            # Wait for minimum epochs to pass before actually stopping
            if epochs_without_improvement >= patience + min_epochs_after_no_improvement:
                print(f"Early stopping triggered at epoch {epoch+1}")
                #break

# Step 8: Load the best model (after early stopping)
model.load_state_dict(torch.load("best_model.pth"))

# Step 9: Evaluate on the test set
model.eval()
predictions = []
true_labels = []

for batch in test_dataloader:
    images, labels = batch
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(images)

    preds = torch.argmax(outputs.logits, dim=1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Classification report
print("Classification Report on Test Set:")
print(classification_report(true_labels, predictions, target_names=full_dataset.classes))

# Step 10: Plot Training and Validation Loss, Accuracy
plt.figure(figsize=(18, 6))

# Plot Loss
plt.subplot(1, 3, 1)
plt.plot(train_loss_history, label='Training Loss')
plt.plot(val_loss_history, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss per Epoch')
plt.legend()

# Plot Training Accuracy
plt.subplot(1, 3, 2)
plt.plot(train_acc_history, label='Training Accuracy')
plt.plot(val_acc_history, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy per Epoch')
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
# Step 11: Dense Plot of All Training Images
# Create a dense grid of all training images
num_train_images = len(train_dataset)
fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(15, 15))  # Adjust the number of rows and columns as needed
axes = axes.flatten()

for i in range(num_train_images):
    image, label = train_dataset[i]
    image = image.cpu().numpy().transpose((1, 2, 0))  # CxHxW -> HxWxC
    image = (image * 0.229 + 0.485)  # Reverse the normalization
    image = np.clip(image, 0, 1)  # Ensure values are between 0 and 1
    
    axes[i].imshow(image)
    axes[i].axis('off')
    axes[i].set_title(full_dataset.classes[label], fontsize=8)

plt.tight_layout()
plt.suptitle('Training Images', fontsize=16)
plt.subplots_adjust(top=0.95)
plt.show()

# Step 12: Dense Plot of All Test Images
# Create a dense grid of all test images
num_test_images = len(test_dataset)
fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(15, 15))  # Adjust the number of rows and columns as needed
axes = axes.flatten()

for i in range(num_test_images):
    image, label = test_dataset[i]
    image = image.cpu().numpy().transpose((1, 2, 0))  # CxHxW -> HxWxC
    image = (image * 0.229 + 0.485)  # Reverse the normalization
    image = np.clip(image, 0, 1)  # Ensure values are between 0 and 1
    
    axes[i].imshow(image)
    axes[i].axis('off')
    axes[i].set_title(full_dataset.classes[label], fontsize=8)

plt.tight_layout()
plt.suptitle('Test Images', fontsize=16)
plt.subplots_adjust(top=0.95)
plt.show()